In [1]:
!pip install selenium
!pip install dateparser
!pip install XlsxWriter

# Libraries
import os
import time
import shutil
import dateparser
import xlsxwriter
import numpy as np
import pandas as pd
from typing import Union
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

# Create folders if not exists
CWD = os.getcwd()
temp_directory = "temp_bddk_files"
directory = 'bddk_files'
dir_list = [directory, temp_directory]
for dir in dir_list:
    if not os.path.exists(dir):
        os.makedirs(dir)

# Configurations
PATH = f'{CWD}/{temp_directory}'
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : PATH}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--headless=new')
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36")


def get_date() -> list:
  URL = 'https://www.bddk.org.tr/bultenhaftalik'
  service = Service()
  driver = webdriver.Chrome(service=service, options=chrome_options)
  time.sleep(3)
  driver.get(URL)
  time.sleep(3)

  date_list = []
  for i in range(1,3):
    date_str = '-'.join(map(str,[driver.find_element("xpath",'//*[@id="Yil"]/option[1]').text] + driver.find_element("xpath",f'//*[@id="Donem"]/option[{i}]').text.split(' ')[0].split('/')))
    date = dateparser.parse(date_str)
    date = date.strftime('%Y-%m-%d')
    date_list.append(date)

  return date_list


def seperate_folders(keyword:str) -> None:
    for filename in os.listdir(PATH):
        if filename.endswith(".xls"):
            folder_name = keyword
            destination_folder = os.path.join(PATH, folder_name)

            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder)

            source_path = os.path.join(PATH, filename)
            destination_path = os.path.join(destination_folder, filename)

            shutil.move(source_path, destination_path)


def scrape_data() -> None:
    tarafDict = {
                10001:500, 10002:501, 10008:502, 10009:503, 10010:504,
                10003:505, 10004:506, 10005:507, 10006:508, 10007:511
                }

    tabloDict = {
                280:'Krediler', 281:'Takipteki Alacaklar', 282:'Menkul Değerler',
                283:'Mevduat', 284:'Diğer Bilanço Kalemleri', 285:'Bilanço Dışı İşlemler',
                }

    tarafNameDict = {
                    500:'Sektör',
                    501:'Mevduat',
                    502:'MevduatKamu',
                    503:'MevduatYabancı',
                    504:'MevduatYerliÖzel',
                    505:'KalkınmaVeYatırım',
                    506:'Katılım',
                    507:'Kamu',
                    508:'Yabancı',
                    511:'YerliÖzel',
                    #512:'KalkınmaYatırım',
                    #513:'MevduatÖzel',
                    }

    URL = 'https://www.bddk.org.tr/bultenhaftalik'
    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    time.sleep(3)
    driver.get(URL)
    time.sleep(3)

    driver.find_element(By.XPATH, '//*[@id="Donem"]').click()
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="Donem"]/option[3]').click()
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="DonemTablosu"]/tbody/tr/td/form/div[4]/button').click()
    time.sleep(3)

    for tar_k, tar_v in tarafDict.items():
        driver.find_element(By.XPATH, '//*[@id="tarafListesiItem-'+str(tar_k)+'"]').click()
        time.sleep(3)
        driver.find_element(By.XPATH, '//*[@id="BTN"]').click()

        for tab_k, tab_v in tabloDict.items():
            driver.find_element(By.XPATH, '//*[@id="tabloListesiItem-'+str(tab_k)+'"]').click()
            time.sleep(3)
            driver.find_element(By.XPATH, '//*[@id="BTN"]').click()
            time.sleep(1)

        seperate_folders(tarafNameDict[tar_v])

    time.sleep(2)
    driver.close()


def remove_unnecessary_files() -> None:
  items = os.listdir(PATH)
  for item in items:
      item_path = os.path.join(PATH, item)
      if os.path.isdir(item_path):
          pass
      else:
          os.remove(item_path)


def rename_xls_files(directory:str=PATH) -> None:
    folder_mapping = {
      'Krediler': '1-Krediler',
      'TakiptekiAlacaklar': '2-TakiptekiAlacaklar',
      'MenkulDeğerler': '3-MenkulDeğerler',
      'Mevduat': '4-Mevduat',
      'DiğerBilan&#231;oKalemleri': '5-DiğerBilançoKalemleri',
      'Bilan&#231;oDışıİşlemler': '6-BilançoDışıİşlemler',
      }

    for folder_name in os.listdir(directory):
        folder_path = os.path.join(directory, folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.lower().endswith('.xls'):
                    for key, value in folder_mapping.items():
                        if key in filename:
                            new_filename = filename.replace(key, value)
                            file_path = os.path.join(folder_path, filename)
                            new_file_path = os.path.join(folder_path, new_filename)
                            os.rename(file_path, new_file_path)


def create_dict() -> dict:
  ordered_dict = {}
  for folder in os.listdir(PATH):
    temp_ordered_list = []
    for files in os.listdir(os.path.join(PATH,folder)):
      temp_ordered_list.append(files)
      temp_ordered_list = sorted(temp_ordered_list, reverse=False)
    ordered_dict[folder] = temp_ordered_list

  return ordered_dict


def clean_dict(dictionary:dict) -> dict:
  cleaned_data = {}

  for key, paths in dictionary.items():
      seen_numbers = set()

      cleaned_paths = []

      for path in paths:
          start_index = path.find("HaftalikBulten _") + len("HaftalikBulten _")
          end_index = path.find("-", start_index)
          if end_index == -1:
              end_index = path.find("_", start_index)

          number = path[start_index:end_index]

          if number not in seen_numbers:
              seen_numbers.add(number)
              cleaned_paths.append(path)

      if cleaned_paths:
        cleaned_data[key] = cleaned_paths

  return cleaned_data


def create_dataframes_dict(cleaned_dict:dict) -> dict:
  df_dict = {}

  for k, v in cleaned_dict.items():
      dfs = []
      for file_path in v:
          full_file_path = os.path.join(PATH, k, file_path)
          data = pd.read_html(full_file_path, encoding='utf8') # thousands='.'
          df = pd.concat(data, ignore_index=True)
          if len(df.columns) == 5:
              columns_with_5 = ['Index', 'Birim: Milyon TL', 'TP', 'YP', 'TOPLAM']
              df.columns = columns_with_5
              df = df.iloc[1:]
              df = df.drop(columns='Index')
          else:
              columns_with_3 = ['Index', 'Birim: Milyon TL', 'TOPLAM']
              df.columns = columns_with_3
              df['TP'] = 0.0
              df['YP'] = 0.0
              df = df[['Index', 'Birim: Milyon TL', 'TP', 'YP', 'TOPLAM']]
              df = df.iloc[1:]
              df = df.drop(columns='Index')
          dfs.append(df)

      if dfs:
          df_dict[k] = pd.concat(dfs, ignore_index=True)

  result_dfs = {}
  for k, df in df_dict.items():
      result_dfs[k] = df.copy()

  return result_dfs


def create_excel(result_dfs:dict) -> None:
  file_name = 'data.xlsx'
  output_path = os.path.join(CWD,directory,file_name)
  with pd.ExcelWriter(output_path) as writer:
    for k, df in result_dfs.items():
      df.to_excel(writer, sheet_name=k, index=False)


def reformat_excel(date:str) -> None:
  # take numbers dynamically
  krediler = ['Krediler'] * 22
  takipteki_alacaklar = ['Takipteki Alacaklar'] * 12
  menkul_degerler = ['Menkul Değerler'] * 13
  mevduat = ['Mevduat'] * 12
  diger_bilanco_kalemleri = ['Diğer Bilanço Kalemleri'] * 15
  bilanco_disi_islemler = ['Bilanço Dışı İşlemler'] * 4

  kalem = krediler + takipteki_alacaklar + menkul_degerler + mevduat + diger_bilanco_kalemleri + bilanco_disi_islemler

  start_values = {
      'Krediler': 101,
      'Takipteki Alacaklar': 201,
      'Menkul Değerler': 301,
      'Mevduat': 401,
      'Diğer Bilanço Kalemleri': 501,
      'Bilanço Dışı İşlemler': 601,
  }

  excel_file_path = os.path.join(CWD, directory, 'data.xlsx')
  excel_data = pd.ExcelFile(excel_file_path)

  sheet_names = excel_data.sheet_names

  sheet_dataframes = {}

  for sheet_name in sheet_names:
      sheet_df = excel_data.parse(sheet_name)
      sheet_df.insert(0, 'Taraf', sheet_name)
      sheet_df.insert(1, 'Kalem', pd.Series(kalem))
      sheet_df.insert(0, 'Metrik',  sheet_df.groupby('Kalem').cumcount() + sheet_df['Kalem'].map(start_values))
      sheet_df['END_DT'] =  date
      sheet_dataframes[sheet_name] = sheet_df

  file_name = f'{date}.xlsx'
  output_excel_path = os.path.join(CWD, directory, file_name)
  with pd.ExcelWriter(output_excel_path) as writer:
      for sheet_name, sheet_df in sheet_dataframes.items():
          sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)


def create_mevduatozel_sheet(date:str) -> None:
  file_name = f'{date}.xlsx'
  excel_path = os.path.join(CWD, directory, file_name)

  df_mevduat_yabanci = pd.read_excel(excel_path, sheet_name='MevduatYabancı')
  df_mevduat_yerliozel = pd.read_excel(excel_path, sheet_name='MevduatYerliÖzel')

  def clean_and_convert(value):
    cleaned_value = str(value).replace('.', '').replace(',', '.')
    return float(cleaned_value)

  df_mevduat_yabanci['TP'] = df_mevduat_yabanci['TP'].apply(clean_and_convert)
  df_mevduat_yabanci['YP'] = df_mevduat_yabanci['YP'].apply(clean_and_convert)
  df_mevduat_yabanci['TOPLAM'] = df_mevduat_yabanci['TOPLAM'].apply(clean_and_convert)

  df_mevduat_yerliozel['TP'] = df_mevduat_yerliozel['TP'].apply(clean_and_convert)
  df_mevduat_yerliozel['YP'] = df_mevduat_yerliozel['YP'].apply(clean_and_convert)
  df_mevduat_yerliozel['TOPLAM'] = df_mevduat_yerliozel['TOPLAM'].apply(clean_and_convert)

  # create new dataframe
  df_mevduat_ozel = pd.DataFrame(columns=df_mevduat_yabanci.columns)

  # create new values with summation
  df_mevduat_ozel['TP'] = df_mevduat_yabanci['TP'] + df_mevduat_yerliozel['TP']
  df_mevduat_ozel['YP'] = df_mevduat_yabanci['YP'] + df_mevduat_yerliozel['YP']
  df_mevduat_ozel['TOPLAM'] = df_mevduat_yabanci['TOPLAM'] + df_mevduat_yerliozel['TOPLAM']

  # fill the missing columns
  df_mevduat_ozel['Metrik'] = df_mevduat_yabanci['Metrik']
  df_mevduat_ozel['Taraf'] = 'MevduatÖzel'
  df_mevduat_ozel['Kalem'] = df_mevduat_yabanci['Kalem']
  df_mevduat_ozel['Birim: Milyon TL'] = df_mevduat_yabanci['Birim: Milyon TL']
  df_mevduat_ozel['END_DT'] = df_mevduat_yabanci['END_DT']

  sheet_name = 'MevduatÖzel'
  with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
      df_mevduat_ozel.to_excel(writer, sheet_name = sheet_name, index=False)


def create_ozel_sheet(date:str) -> None:
  file_name = f'{date}.xlsx'
  excel_path = os.path.join(CWD, directory, file_name)

  df_yabanci = pd.read_excel(excel_path, sheet_name='Yabancı')
  df_yerliozel = pd.read_excel(excel_path, sheet_name='YerliÖzel')

  def clean_and_convert(value):
    cleaned_value = str(value).replace('.', '').replace(',', '.')
    return float(cleaned_value)

  df_yabanci['TP'] = df_yabanci['TP'].apply(clean_and_convert)
  df_yabanci['YP'] = df_yabanci['YP'].apply(clean_and_convert)
  df_yabanci['TOPLAM'] = df_yabanci['TOPLAM'].apply(clean_and_convert)

  df_yerliozel['TP'] = df_yerliozel['TP'].apply(clean_and_convert)
  df_yerliozel['YP'] = df_yerliozel['YP'].apply(clean_and_convert)
  df_yerliozel['TOPLAM'] = df_yerliozel['TOPLAM'].apply(clean_and_convert)

  # create new dataframe
  df_ozel = pd.DataFrame(columns=df_yabanci.columns)

  # create new values with summation
  df_ozel['TP'] = df_yabanci['TP'] + df_yerliozel['TP']
  df_ozel['YP'] = df_yabanci['YP'] + df_yerliozel['YP']
  df_ozel['TOPLAM'] = df_yabanci['TOPLAM'] + df_yerliozel['TOPLAM']

  # fill the missing columns
  df_ozel['Metrik'] = df_yabanci['Metrik']
  df_ozel['Taraf'] = 'Özel'
  df_ozel['Kalem'] = df_yabanci['Kalem']
  df_ozel['Birim: Milyon TL'] = df_yabanci['Birim: Milyon TL']
  df_ozel['END_DT'] = df_yabanci['END_DT']

  sheet_name = 'Özel'
  with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
      df_ozel.to_excel(writer, sheet_name = sheet_name, index=False)


def transform_numeric_columns(date:str) -> None:
  def clean_and_convert(value):
      cleaned_value = str(value).replace('.', '').replace(',', '.')
      return float(cleaned_value)


  excel_file_path = os.path.join(CWD, directory, f'{date}.xlsx')
  excel_data = pd.ExcelFile(excel_file_path)

  sheet_names = excel_data.sheet_names
  sheet_dataframes = {}

  for sheet_name in sheet_names:
    sheet_df = excel_data.parse(sheet_name)
    sheet_df['TP'] = sheet_df['TP'].apply(clean_and_convert)
    sheet_df['YP'] = sheet_df['YP'].apply(clean_and_convert)
    sheet_df['TOPLAM'] = sheet_df['TOPLAM'].apply(clean_and_convert)
    sheet_dataframes[sheet_name] = sheet_df

  with pd.ExcelWriter(excel_file_path) as writer:
    for sheet_name, sheet_df in sheet_dataframes.items():
      sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)


def create_weekly_report(date_list:list):
  current_date, previous_date = date_list

  filename_1, filename_2 = f'{current_date}.xlsx', f'{previous_date}.xlsx'
  file_1, file_2 = os.path.join(CWD,directory,filename_1), os.path.join(CWD,directory,filename_2)

  current_week_dataframes = pd.read_excel(file_2, sheet_name=None)
  previous_week_dataframes = pd.read_excel(file_1, sheet_name=None)

  columns_to_add = ['TP', 'YP', 'TOPLAM', 'END_DT']

  updated_dataframes = {}

  for sheet_name in previous_week_dataframes.keys():
    previous_week_df = previous_week_dataframes[sheet_name]
    current_week_df = current_week_dataframes[sheet_name]

    current_week_selected = current_week_df[columns_to_add]

    updated_df = pd.concat([previous_week_df, current_week_selected], axis=1)

    updated_dataframes[sheet_name] = updated_df

  file_name = 'Sektör_Haftalık_Data.xlsx'
  with pd.ExcelWriter(os.path.join(CWD,directory,file_name)) as writer:
      for sheet_name, updated_df in updated_dataframes.items():
          updated_df.to_excel(writer, sheet_name=sheet_name, index=False)


def run():
    date_list = get_date()  # (current_date, previous_date)

    for date in date_list:
      print(f'{date} tarihli veri çekiliyor..')
      scrape_data()
      remove_unnecessary_files()
      rename_xls_files()
      ordered_dict = create_dict()
      cleaned_dict = clean_dict(dictionary=ordered_dict)
      result_dfs = create_dataframes_dict(cleaned_dict=cleaned_dict)
      create_excel(result_dfs=result_dfs)
      reformat_excel(date=date)
      create_mevduatozel_sheet(date=date)
      create_ozel_sheet(date=date)
      transform_numeric_columns(date=date)
      shutil.rmtree(PATH, ignore_errors=True)
      print(f'{date} tarihli veri başarıyla çekildi!')

    create_weekly_report(date_list=date_list)
run()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 2.7 MB/s eta 0:00:00
2023-08-11 tarihli veri çekiliyor..
2023-08-11 tarihli veri başarıyla çekildi!
2023-08-04 tarihli veri çekiliyor..
2023-08-04 tarihli veri başarıyla çekildi!


In [ ]:
# Seperate dataframes
df_500 = weekly_df[weekly_df['TARAF'] == 'Sektör']
df_501 = weekly_df[weekly_df['TARAF'] == 'Mevduat']
df_506 = weekly_df[weekly_df['TARAF'] == 'Mevduat - Kamu']
df_507 = weekly_df[weekly_df['TARAF'] == 'Mevduat - Yabancı']
df_508 = weekly_df[weekly_df['TARAF'] == 'Mevduat - Yerli Özel']
df_512 = weekly_df[weekly_df['TARAF'] == 'Kalkınma ve Yatırım']
df_511 = weekly_df[weekly_df['TARAF'] == 'Katılım']
df_502 = weekly_df[weekly_df['TARAF'] == 'Kamu']
df_504 = weekly_df[weekly_df['TARAF'] == 'Yabancı']
df_503 = weekly_df[weekly_df['TARAF'] == 'Yerli Özel']

# df_ozel = weekly_df[weekly_df['TARAF'] == 'Özel']
# df_mevduat_ozel = weekly_df[weekly_df['TARAF'] == 'Mevduat Özel']

# Sorted Dataframe List
dataframes = [df_500,df_501,df_502,df_503,df_504,
           df_506,df_507,df_508,df_511,df_512]

def reformat_dataframe(df:pd.DataFrame) -> pd.DataFrame:
  # Create Missing Columns
  df['METRIK_IZLM_DEGER'] = df['METRIK_DEGER']
  df['IS_AKIS_NO'] = 26

  # Relocate Columns with Correct Order
  df = df[['METRIK_NO', 'END_DT', 'TL_YP_KOD', 'METRIK_DEGER', 'METRIK_IZLM_DEGER', 'BNK_KRM_KOD', 'IS_AKIS_NO']]

  # Convert Datetime to Correct Format
  df['END_DT'] = pd.to_datetime(df['END_DT'], errors='coerce')
  df['END_DT'] = df['END_DT'].dt.strftime('%m/%d/%Y')

  # Sort Values with TL_YP_KOD and METRIK_NO Columns
  df['TL_YP_KOD'] = pd.Categorical(df['TL_YP_KOD'], ordered=True,
                                       categories= ['T', 'Y', 'P'])
  df = df.sort_values(by=['TL_YP_KOD','METRIK_NO'])
  return df

reformated_dataframes = []

for dataframe in dataframes:
  reformated_dataframes.append(reformat_dataframe(dataframe))

df = pd.concat(dataframes)